#### **E. Contextual Embeddings**

So far, the embeddings that we have learned are described as `static`. This means that the vector representation for a given token **regardless** of the `context` in which the token occurs.

Words can have more than one meaning depending on the context they appear:

1. Manila is the `capital` of the Philippines.
2. The most important advice anyone can receive in trading the market is risk management - protect your `capital`, profits just follow.

Static word embeddings cannot capture this distinction because words will always have the same vector representation. The values of the `n-dimensional` vector remains the same.

We would like to encode the context by which a word occurs to draw out these differences in meaning, using an architecture called **`Transformers`**.

A **`Transformer`** is a neural network architecture that is capable of encoding information about the context which a token occurs. It improved performance for traditional natural language processing tasks such as part-of-speech tagging and syntactic parsing, but it is `computationally heavy`.

In [ ]:
import spacy

In [ ]:
# Load a Transformer-based language model
nlp_trf = spacy.load('en_core_web_trf')

On the surface, a *Language* object that contains a Transformer-based model looks and works just like any other language model in spaCy.

However, if we look under the hood of the *Language* object under `nlp_trf` using `pipeline`, we will see that the first component in the processing pipeline is a Transformer.

In [ ]:
nlp_trf.pipeline

In [ ]:
# Feed an example sentence to the model
example_doc = nlp_trf('The most important advice anyone can receive in trading the market is risk management - protect your capital, profits just follow')